<a href="https://colab.research.google.com/github/SanjayBhargavKudupudi/DATA_MINING_4/blob/main/Beam/Beam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install apache-beam[interactive]


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 17.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 44.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 671.3/671.3 kB 44.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 56.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.4/308.4 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.0/807.0 kB 56.8 M

In [2]:
import apache_beam as beam

with beam.Pipeline() as pipeline:
    (
        pipeline
        | "Read lines" >> beam.Create(["Hello Beam", "Apache Beam is awesome"])
        | "Print lines" >> beam.Map(print)
    )


Hello Beam
Apache Beam is awesome


In [3]:
#@title Composite Transform
class SplitAndLength(beam.PTransform):
    def expand(self, pcoll):
        return (
            pcoll
            | "Split" >> beam.FlatMap(lambda x: x.split(" "))
            | "Length" >> beam.Map(lambda x: (x, len(x)))
        )

with beam.Pipeline() as pipeline:
    (
        pipeline
        | "Read lines" >> beam.Create(["Hello Beam", "Apache Beam is cool"])
        | "Split and length" >> SplitAndLength()
        | "Print output" >> beam.Map(print)
    )


('Hello', 5)
('Beam', 4)
('Apache', 6)
('Beam', 4)
('is', 2)
('cool', 4)


In [6]:
# @title Windowing
with beam.Pipeline() as pipeline:
    (
        pipeline
        | beam.Create([
            beam.window.TimestampedValue((1, 1), 1),
            beam.window.TimestampedValue((2, 1), 2),
            beam.window.TimestampedValue((3, 5), 5),
            beam.window.TimestampedValue((4, 7), 7)
        ])
        | beam.WindowInto(beam.window.FixedWindows(2))
        | beam.Map(print)
    )


(1, 1)
(2, 1)
(3, 5)
(4, 7)


In [8]:
#@title Triggers
with beam.Pipeline() as pipeline:
    (
        pipeline
        | beam.Create([
            beam.window.TimestampedValue((1, 1), 1),
            beam.window.TimestampedValue((2, 1), 2),
            beam.window.TimestampedValue((3, 5), 5),
            beam.window.TimestampedValue((4, 7), 7)
        ])
        | beam.WindowInto(beam.window.FixedWindows(2), trigger=beam.transforms.trigger.AfterCount(2), accumulation_mode=beam.transforms.trigger.AccumulationMode.DISCARDING)
        | beam.Map(print)
    )


(1, 1)
(2, 1)
(3, 5)
(4, 7)


In [9]:
# @title Pardo
class ProcessWords(beam.DoFn):
    def process(self, element):
        if "Beam" in element:
            yield element

with beam.Pipeline() as pipeline:
    (
        pipeline
        | "Read lines" >> beam.Create(["Hello Beam", "Apache is cool"])
        | "Filter Beam words" >> beam.ParDo(ProcessWords())
        | "Print output" >> beam.Map(print)
    )


Hello Beam
